# Fake news detection using neural networks and linear regression

In [ ]:
#ide menjen az osszes package
!pip install contractions


# Downloading dataset
The dataset: [GonzaloA/fake_news](https://huggingface.co/datasets/GonzaloA/fake_news)

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/GonzaloA/fake_news/" + splits["train"])

df.head()

,Unnamed: 0,title,text,label
0,0,‘Maury’ Show Official Facebook Posts F*CKED U...,Maury is perhaps one of the trashiest shows on...,0
1,1,Trump’s Favorite News Channel Tries To Soothe...,"Yesterday, after the father of one of the UCLA...",0
2,2,"Russia warns Iraq, Kurds not to destabilize Mi...",MOSCOW (Reuters) - Russia on Wednesday warned ...,1
3,3,WATCH STEVE SCALISE Throw A Strike At The Nati...,"House Majority Whip Steve Scalise (R., La.) th...",0
4,4,Trump Will HATE What Stephen Colbert Just Did...,It can be said that Late Show host Stephen Col...,0


# Text preprocessing
Removing stopwords, html tags and special characters.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

def clean_text_advanced(text):
  text = text.lower()
  text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
  text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
  text = text.strip()
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
  stop_words = set(stopwords.words('english'))
  words = text.split()
  filtered_words = [word for word in words if word not in stop_words]
  text = " ".join(filtered_words)
  return text

df['text'] = df['text'].apply(clean_text_advanced)
df['title'] = df['title'].apply(clean_text_advanced)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,title,text,label
0,0,maury show official facebook posts fcked capti...,maury perhaps one trashiest shows television t...,0
1,1,trumps favorite news channel tries soothe batt...,yesterday father one ucla players arrested chi...,0
2,2,russia warns iraq kurds destabilize middle eas...,moscow reuters russia wednesday warned iraq ku...,1
3,3,watch steve scalise throw strike nationals bas...,house majority whip steve scalise r la threw s...,0
4,4,trump hate stephen colbert pure comedy genius ...,said late show host stephen colbert clearly gi...,0


## Text cleaning validation

In [ ]:
#check text cleaning

def check_text_cleaning(text):
  """Checks if the text is in lowercase and HTML tags are removed."""
  if not isinstance(text, str):
    return False

  if text.islower() and "<" not in text and ">" not in text:
    return True
  else:
    return False

# Apply the function to the 'text' column and check if all rows are true
cleaned_status = df['text'].apply(check_text_cleaning).all()

print(cleaned_status)

False


Lemmatization for further simplification of the text.

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
  words = text.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
  return " ".join(lemmatized_words)


df['text'] = df['text'].apply(stem_and_lemmatize)
df['title'] = df['title'].apply(stem_and_lemmatize)
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Unnamed: 0,title,text,label
0,0,mauri show offici facebook post fcked caption ...,mauri perhap one trashiest show televis today ...,0
1,1,trump favorit news channel tri sooth batter eg...,yesterday father one ucla player arrest china ...,0
2,2,russia warn iraq kurd destabil middl east kurd...,moscow reuter russia wednesday warn iraq kurd ...,1
3,3,watch steve scalis throw strike nation basebal...,hous major whip steve scalis r la threw strike...,0
4,4,trump hate stephen colbert pure comedi geniu v...,said late show host stephen colbert clearli gi...,0


## Validating text preprocessing results

In [ ]:
# prompt: check if stemming and lemmatizing are done correctly

import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ... (rest of your existing code)

# Improved check_text_cleaning function
def check_text_cleaning(text):
    """Checks if stemming and lemmatization were applied correctly."""
    if not isinstance(text, str):
        return False

    # Check for stemmed words (heuristic)
    words = text.split()
    for word in words:
        if len(word) > 3 and word.endswith("ing") :  # Example heuristic
            return False
    return True

# Apply the function to the 'text' column and check if all rows are true
cleaned_status = df['text'].apply(check_text_cleaning).all()

cleaned_status

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


False

In [ ]:
x = df['text'].values
y = df['label'].values

x.shape, y.shape

((24353,), (24353,))































































































































































































































































































# Splitting dataset into training, testing and validation splits

In [ ]:
# prompt: split the ds text and labels to train eval and test

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data into training and temporary sets (80% train, 20% temp)
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)

# Split temporary set into validation and test sets (50% eval, 50% test)
x_eval, x_test, y_eval, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Now you have x_train, y_train, x_eval, y_eval, x_test, y_test
print("Train data shape:", x_train.shape, y_train.shape)
print("Eval data shape:", x_eval.shape, y_eval.shape)
print("Test data shape:", x_test.shape, y_test.shape)

Train data shape: (19482,) (19482,)
Eval data shape: (2435,) (2435,)
Test data shape: (2436,) (2436,)
Train data shape after vectorization: (19482, 1000)
Eval data shape after vectorization: (2435, 1000)
Test data shape after vectorization: (2436, 1000)


# Vectorisation
Counter vectorizer

In [ ]:
# prompt: apply countvectorizer to the x train test and val

vectorizer = CountVectorizer(max_features=1000)

# Fit and transform the training data
x_train_vec = vectorizer.fit_transform(x_train)

# Transform the validation and test data using the same vectorizer
x_eval_vec = vectorizer.transform(x_eval)
x_test_vec = vectorizer.transform(x_test)

print("Train data shape after vectorization:", x_train_vec.shape)
print("Eval data shape after vectorization:", x_eval_vec.shape)
print("Test data shape after vectorization:", x_test_vec.shape)

TF-IDF vectorisation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the training data
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)

# Transform the validation and test data using the same vectorizer
x_eval_tfidf = tfidf_vectorizer.transform(x_eval)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

print("Train data shape after TF-IDF:", x_train_tfidf.shape)
print("Eval data shape after TF-IDF:", x_eval_tfidf.shape)
print("Test data shape after TF-IDF:", x_test_tfidf.shape)

Train data shape after TF-IDF: (19482, 1000)
Eval data shape after TF-IDF: (2435, 1000)
Test data shape after TF-IDF: (2436, 1000)


# Creating and training models
Neural network

In [ ]:
# prompt: generate a classificator neural network with two layer and apply to countervectorized data
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train_vec.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train_vec, y_train, epochs=10, batch_size=32, validation_data=(x_eval_vec, y_eval))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_vec, y_test)
print('Test accuracy:', accuracy)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9146 - loss: 0.2907 - val_accuracy: 0.9704 - val_loss: 0.1014
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9817 - loss: 0.0625 - val_accuracy: 0.9725 - val_loss: 0.0885
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9885 - loss: 0.0349 - val_accuracy: 0.9630 - val_loss: 0.1545
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9908 - loss: 0.0282 - val_accuracy: 0.9737 - val_loss: 0.1115
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9945 - loss: 0.0150 - val_accuracy: 0.9733 - val_loss: 0.1373
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9958 - loss: 0.0121 - val_accuracy: 0.9663 - val_loss: 0.1642
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9967 - loss: 0.0122 - val_accuracy: 0.9725 - val_loss: 0.1559
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9981 - loss: 0.0062 - val_accuracy: 0.

In [ ]:
# prompt: generate a neural network with two layer apply tfidf to it

# Define the model
model_tfidf = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(x_train_tfidf.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model_tfidf.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model_tfidf.fit(x_train_tfidf, y_train, epochs=10, batch_size=32, validation_data=(x_eval_tfidf, y_eval))

# Evaluate the model on the test set
loss, accuracy = model_tfidf.evaluate(x_test_tfidf, y_test)
print('Test accuracy with TF-IDF:', accuracy)


Epoch 1/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8738 - loss: 0.3866 - val_accuracy: 0.9626 - val_loss: 0.1248
Epoch 2/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9697 - loss: 0.0967 - val_accuracy: 0.9676 - val_loss: 0.1081
Epoch 3/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9782 - loss: 0.0647 - val_accuracy: 0.9671 - val_loss: 0.1067
Epoch 4/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9849 - loss: 0.0450 - val_accuracy: 0.9671 - val_loss: 0.1175
Epoch 5/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9909 - loss: 0.0263 - val_accuracy: 0.9667 - val_loss: 0.1262
Epoch 6/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9970 - loss: 0.0140 - val_accuracy: 0.9684 - val_loss: 0.1424
Epoch 7/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9984 - loss: 0.0072 - val_accuracy: 0.9630 - val_loss: 0.1734
Epoch 8/10
609/609 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9984 - loss: 0.0063 - val_accuracy: 0.

## Evaluating the performance of the two differently vectorised neural networks

In [ ]:
# prompt: evaluate the models

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_vec, y_test)
print('Test accuracy with CountVectorizer:', accuracy)

loss, accuracy = model_tfidf.evaluate(x_test_tfidf, y_test)
print('Test accuracy with TF-IDF:', accuracy)

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9766 - loss: 0.1322
Test accuracy with CountVectorizer: 0.9766010046005249
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9762 - loss: 0.1022
Test accuracy with TF-IDF: 0.9766010046005249


Testing the models with a custom input

In [ ]:
# prompt: apply the model for custom data

# Sample custom text
custom_text = "MOSCOW (Reuters) - Russia on Wednesday warned Iraq and the Kurds against taking any steps that might destabilize the Middle East after a Kurdish independence referendum, encouraging both sides to hold talks to find a solution within the framework of a single Iraqi state. The Russian Foreign Ministry, in the same statement, also said that while Moscow respected the Kurds national ambitions it favored preserving the territorial integrity of Iraq."

# Preprocess the custom text
custom_text = clean_text_advanced(custom_text)
custom_text = stem_and_lemmatize(custom_text)

# Vectorize the custom text using the same vectorizer used for training
custom_text_vec = vectorizer.transform([custom_text])

# Make a prediction using the trained model
prediction = model.predict(custom_text_vec)

# Get the predicted class (the class with the highest probability)
predicted_class = prediction.argmax()

print("Predicted class:", predicted_class)

# You might want to map the predicted class to the actual label if you have a mapping between class indices and labels
# For example:
# label_mapping = {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
# predicted_label = label_mapping[predicted_class]
# print("Predicted label:", predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted class: 1


Exporting the trained models into files.

In [ ]:
# prompt: save the model in a file and the vectorizer

import joblib

# Save the model
model.save('fake_news_model.keras')
model_tfidf.save('fake_news_tfidf.keras')

# Save the vectorizer
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
# prompt: load the model back

import tensorflow as tf

loadedmodel = tf.keras.models.load_model('fake_news_model.keras')
loadedmodeltf = tf.keras.models.load_model('fake_news_tfidf.keras')

In [ ]:
# prompt: test loadedmodel with text input from console

import numpy as np

while True:
    user_input = input("Enter text to classify (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # Preprocess the user input
    user_input = clean_text_advanced(user_input)
    user_input = stem_and_lemmatize(user_input)
    user_input_vec = vectorizer.transform([user_input])

    # Make a prediction
    prediction = loadedmodel.predict(user_input_vec)
    predicted_class = np.argmax(prediction)  # Use argmax for multi-class

    print("Predicted class:", predicted_class)

Enter text to classify (or type 'exit' to quit): In a recent interview with CNN, former senator and Republican presidential candidate Rick Santorum hinted that President Trump s time so far in office would be more successful and that he still has the ability to become a better leader if he makes one simple change Just stop tweeting.During Sunday morning s edition of State of the Union with Dana Bash, Santorum made his point clear in absolutely no uncertain terms. If you tweet every day and complain about the media and complain about how you are being treated, you re going to be sidetracked and you re not going to get your deals done. That s the message, Santorum said, almost trying to channel the president directly by staring straight into the camera and using terms that Trump could relate to in order to get his advice across.Donald Trump has just concluded his first ever overseas trip as President of the United States, a nine-day journey into the Middle East, and during that time he h

In [ ]:
# prompt: test loadedmodel with text input from console

import numpy as np

while True:
    user_input = input("Enter text to classify (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # Preprocess the user input
    user_input = clean_text_advanced(user_input)
    user_input = stem_and_lemmatize(user_input)
    user_input_tf = tfidf_vectorizer.transform([user_input])

    # Make a prediction
    prediction = loadedmodeltf.predict(user_input_tf)
    predicted_class = np.argmax(prediction)  # Use argmax for multi-class

    print("Predicted class:", predicted_class)

Enter text to classify (or type 'exit' to quit): In a recent interview with CNN, former senator and Republican presidential candidate Rick Santorum hinted that President Trump s time so far in office would be more successful and that he still has the ability to become a better leader if he makes one simple change Just stop tweeting.During Sunday morning s edition of State of the Union with Dana Bash, Santorum made his point clear in absolutely no uncertain terms. If you tweet every day and complain about the media and complain about how you are being treated, you re going to be sidetracked and you re not going to get your deals done. That s the message, Santorum said, almost trying to channel the president directly by staring straight into the camera and using terms that Trump could relate to in order to get his advice across.Donald Trump has just concluded his first ever overseas trip as President of the United States, a nine-day journey into the Middle East, and during that time he h

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Predicted class: 0


KeyboardInterrupt: Interrupted by user

# Creating and training models
Linear regression

In [ ]:
# prompt: train a model with linear regression

from sklearn.linear_model import LinearRegression

# Assuming x_train_vec and y_train are your training data
# Replace with your actual data

# Create and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(x_train_vec, y_train)


# Make predictions on the test set
y_pred = linear_model.predict(x_test_vec)

# Evaluate the model (example using mean squared error)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# You can also use other metrics like R-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Squared Error: 0.07551629156340117
R-squared: 0.69620156217292


Saving the model to a file.

In [ ]:
# prompt: save the linear_model

# Save the linear model
joblib.dump(linear_model, 'linear_model.pkl')

['linear_model.pkl']

Evaluating the linear regression model.

In [ ]:
# prompt: train a  new model with linear regression on the x_test_tfidf

# Create and train the linear regression model for TF-IDF features
linear_model_tfidf = LinearRegression()
linear_model_tfidf.fit(x_train_tfidf, y_train)

# Make predictions on the test set using TF-IDF features
y_pred_tfidf = linear_model_tfidf.predict(x_test_tfidf)

# Evaluate the model (example using mean squared error)
mse_tfidf = mean_squared_error(y_test, y_pred_tfidf)
print(f"Mean Squared Error (TF-IDF): {mse_tfidf}")

# You can also use other metrics like R-squared
r2_tfidf = r2_score(y_test, y_pred_tfidf)
print(f"R-squared (TF-IDF): {r2_tfidf}")

# Save the linear model for TF-IDF features
joblib.dump(linear_model_tfidf, 'linear_model_tfidf.pkl')

Mean Squared Error (TF-IDF): 0.04649066169628715
R-squared (TF-IDF): 0.8129702862193464


['linear_model_tfidf.pkl']

Title: Fake News Detection using Machine Learning

Introduction (1 minute)

Problem: Fake news is a widespread problem that can have serious consequences.
Solution: We can use machine learning to automatically detect fake news articles.
Approach: We will train a neural network model on a dataset of labeled news articles.
Data Preprocessing (2 minutes)

Data Source: The code uses the GonzaloA/fake_news dataset from Hugging Face Datasets.
Cleaning: The clean_text_advanced function is used to clean the text data:
Lowercasing
Removing punctuation, HTML tags, and extra whitespace
Removing stop words
Stemming and Lemmatization:
stem_and_lemmatize function is used to reduce words to their base forms using Porter Stemmer and WordNet Lemmatizer.
This helps to improve the accuracy of the model.
Verification:
The check_text_cleaning function checks if cleaning and normalization were successful.
Feature Extraction (2 minutes)

CountVectorizer:
Converts text into numerical vectors based on word frequencies.
Creates a vocabulary of the most frequent words.
TF-IDF Vectorizer:
Similar to CountVectorizer, but also considers the importance of words in the entire dataset.
Gives higher weight to words that are frequent in a document but rare in the corpus.
Model Training and Evaluation (3 minutes)

Neural Network:
A simple two-layer neural network is used for classification.
Activation functions: ReLU for hidden layers, softmax for output layer.
Loss function: Sparse categorical cross-entropy.
Optimizer: Adam.
Training:
The model is trained on the training set and validated on the validation set.
The number of epochs and batch size are specified.
Evaluation:
The model's performance is evaluated on the test set using accuracy as a metric.
Saving and Loading the Model (1 minute)

Saving:
The trained model and the vectorizer are saved to files using model.save and joblib.dump.
Loading:
The saved model and vectorizer can be loaded back using tf.keras.models.load_model and joblib.load.
Custom Data Prediction (1 minute)

Input:
The user can input custom text for classification.
Preprocessing:
The input text is preprocessed using the same functions used for training data.
Prediction:
The loaded model is used to predict the class of the input text.
Output:
The predicted class is displayed to the user.
Conclusion

The code demonstrates how to build a simple fake news detection system using machine learning.
Further improvements can be made by experimenting with different models, hyperparameters, and feature engineering techniques.
This project highlights the potential of AI in addressing real-world problems like fake news.